# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](/docs/how_to/tool_calling) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [1]:
%pip install --upgrade --quiet langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.2 MB/s eta 0:00:00


In [4]:
!pip install ollama openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [2]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.3 MB/s eta 0:00:00


In [3]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 783.8 kB/s eta 0:00:00


In [14]:
!pip install flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cache

If you'd like to use LangSmith, uncomment the below:

In [5]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](/docs/integrations/chat/), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](/docs/how_to/tool_calling) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](/docs/integrations/chat/ollama/).

In [6]:
!curl -fsSL https://ollama.com/install.sh | sh


>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [9]:
!rm /content/myscript.sh
!echo '#!/bin/bash' > /content/myscript.sh
!echo 'ollama serve' >> /content/myscript.sh
# Make the script executable
!chmod +x /content/myscript.sh
!nohup /content/myscript.sh &

nohup: appending output to 'nohup.out'


In [14]:
!ollama pull phi3:medium


pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest 
pulling 7a3cc4804c64... 100% ▕▏ 7.9 GB                         
pulling fa8235e5b48f... 100% ▕▏ 1.1 KB                         
pulling 542b217f179c... 100% ▕▏  148 B                         
pulling 8dde1baf1db0... 100% ▕▏   78 B                         
pulling c266f81b82cd... 100% ▕▏  483 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [71]:
import ollama

# Setting up the model, enabling streaming responses, and defining the input messages
ollama_response = ollama.chat(model='phi3:medium', messages=[
   {
     'role': 'system',
     'content': 'You are a helpful assistant with sound mathematical knowledge.',
   },
   {
     'role': 'user',
     'content': 'Explain to me the Pythagorean theorem.',
   },

])
# Printing out of the generated response
print(ollama_response['message']['content'])

 The Pythagoreebian Theorem is a fundamental principle in geometry that describes the relationship between the sides of a right-angled triangle (a triangle where one angle measures 90 degrees). It states that, for any such triangle with perpendicular sides labeled 'a' and 'b', and hypotenuse (the longest side opposite to the right angle) labeled as 'c,'

The square of the length of the hypotenuse is equal to the sum of the squares of the other two sides. Mathematically, this can be expressed using the following equation:

a² + b² = c²

To use the Pythagorean theorem in practice, simply follow these steps:

1. Identify a right-angled triangle and label its perpendicular sides as 'a' and 'b,' and the hypotenuse as 'c.'
2. Square the lengths of both sides 'a' and 'b', then add their squares together to obtain the value (a² + b²).
3. Find the square root of this sum, which gives you the length of side 'c'.

Conversely, if you know the lengths of two sides, you can use the Pythagorean theor

## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](/docs/how_to/custom_tools).

In [49]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [21]:
multiply.invoke({"x": 4, "y": 5})

20.0

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [22]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.


In [63]:
from langchain.schema.runnable import Runnable
import ollama

class OllamaLLM(Runnable):
    def __init__(self, model_name):
        self.model_name = model_name

    def invoke(self, input, config=None):
        # Handle ChatPromptValue object
        if hasattr(input, "to_string"):
            user_input = input.to_string()
        else:
            user_input = input.get("input", "")

        response = ollama.chat(model=self.model_name, messages=[
            {
                'role': 'system',
                'content': f"""\
You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with the first key corresponding to 'x' and the second key corresponding to 'y'.
"""
            },
            {
                'role': 'user',
                'content': user_input,
            },
        ])
        return response['message']['content']


In [64]:
from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(
    """
User input: {input}
"""
)

# Instantiate the OllamaLLM with the desired model name
model = OllamaLLM(model_name='phi3:medium')

chain = prompt | model
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

 {
  "name": "add",
  "arguments": {
    "x": 3,
    "y": 1132
  }
}


## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [65]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'x': 13, 'y': 4}}

:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [66]:
from typing import Any, Dict, Optional, TypedDict
import logging
from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)


Let's test this out 🧪!

In [67]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [69]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})


53.83784653

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [70]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).